# Problem Statement

Given Subset of Amazon_Transaction data for year 2019.Recommend most commonly purchased products together, if user has given some input product.To limit the scope, you can target only a city say San Francisco and not the whole US.


Learnings:<br>
How to handle huge dataset<br>
How to deal with Textual data<br>
How to build prototype recommender systembr> <br>
How to make recommendations more efficient<br>

In [1]:
import pandas as pd

# Reading Data

In [2]:
from google.colab import files
uploaded = files.upload()

Saving amazon_transaction.csv to amazon_transaction.csv


In [4]:
df = pd.read_csv("amazon_transaction.csv", delimiter=',')
df.dropna(inplace=True)
df.head(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
6,176562,USB-C Charging Cable,1,11.95,04/29/19 13:03,"381 Wilson St, San Francisco, CA 94016"
7,176563,Bose SoundSport Headphones,1,99.99,04/02/19 07:46,"668 Center St, Seattle, WA 98101"
8,176564,USB-C Charging Cable,1,11.95,04/12/19 10:58,"790 Ridge St, Atlanta, GA 30301"
9,176565,Macbook Pro Laptop,1,1700,04/24/19 10:38,"915 Willow St, San Francisco, CA 94016"
10,176566,Wired Headphones,1,11.99,04/08/19 14:05,"83 7th St, Boston, MA 02215"


In [5]:
print(df.shape)

(186305, 6)


In [6]:
# Get only those products whose' address is in San Francisco
def filter_data_based_on_city(address):
  city = "San Francisco"
  if city in address: 
    return address
  else:
    return None

In [7]:
df['Purchase Address'] = df['Purchase Address'].apply(filter_data_based_on_city)
df['Purchase Address'] = df['Purchase Address'].astype(str)

df = df.loc[df['Purchase Address'] != "None"]

In [8]:
print(df.shape)
df.head(10)

(44732, 6)


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
6,176562,USB-C Charging Cable,1,11.95,04/29/19 13:03,"381 Wilson St, San Francisco, CA 94016"
9,176565,Macbook Pro Laptop,1,1700,04/24/19 10:38,"915 Willow St, San Francisco, CA 94016"
17,176573,USB-C Charging Cable,1,11.95,04/27/19 18:41,"214 Chestnut St, San Francisco, CA 94016"
29,176584,Flatscreen TV,1,300,04/24/19 20:39,"936 Church St, San Francisco, CA 94016"
32,176586,AAA Batteries (4-pack),2,2.99,04/10/19 17:00,"365 Center St, San Francisco, CA 94016"
33,176586,Google Phone,1,600,04/10/19 17:00,"365 Center St, San Francisco, CA 94016"
41,176594,Wired Headphones,1,11.99,04/17/19 23:04,"63 Maple St, San Francisco, CA 94016"
44,176597,USB-C Charging Cable,1,11.95,04/19/19 13:39,"681 11th St, San Francisco, CA 94016"
45,176598,AAA Batteries (4-pack),2,2.99,04/21/19 20:33,"294 Park St, San Francisco, CA 94016"
50,176603,USB-C Charging Cable,1,11.95,04/01/19 07:46,"858 6th St, San Francisco, CA 94016"


In [9]:
df.describe()
#There are 19 unique products.

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
count,44732,44732,44732,44732,44732,44732
unique,42898,19,9,23,40554,28324
top,160873,USB-C Charging Cable,1,11.95,11/19/19 18:25,"193 Forest St, San Francisco, CA 94016"
freq,5,5368,40608,5368,5,9


In [10]:
print(df['Product'].unique())

['USB-C Charging Cable' 'Macbook Pro Laptop' 'Flatscreen TV'
 'AAA Batteries (4-pack)' 'Google Phone' 'Wired Headphones' 'LG Dryer'
 'iPhone' 'Bose SoundSport Headphones' 'AA Batteries (4-pack)'
 'Lightning Charging Cable' 'Vareebadd Phone' 'Apple Airpods Headphones'
 '20in Monitor' '34in Ultrawide Monitor' 'LG Washing Machine'
 '27in 4K Gaming Monitor' 'ThinkPad Laptop' '27in FHD Monitor']


In [11]:
print(df.columns)

Index(['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address'],
      dtype='object')


In [12]:
df = df[['Order ID', 'Product']]
df.head(10)

,Order ID,Product
6,176562,USB-C Charging Cable
9,176565,Macbook Pro Laptop
17,176573,USB-C Charging Cable
29,176584,Flatscreen TV
32,176586,AAA Batteries (4-pack)
33,176586,Google Phone
41,176594,Wired Headphones
44,176597,USB-C Charging Cable
45,176598,AAA Batteries (4-pack)
50,176603,USB-C Charging Cable


In [201]:
def merge_products_with_same_order_id():
      # Creating a deep copy of the original data.
  df1 = df.copy()
      # Group by order ID and use an aggregate function to join two products into a single string.
  # print(df1.head(4))
  df1 = df1.groupby('Order ID').agg(lambda x: ', '.join(x)).reset_index()
      # Remove individual values. i.e. without any pairs. We need only pairs right now and store it in a different dataframe.
  df_final = df1[df1['Product'].str.contains(",")]
      # Group by products now, There are 268 unique combinations. The count will be stored in Order ID column.
  df_final = df_final.groupby('Product')['Order ID'].nunique().reset_index()
  return df_final         # This returns the grouped dataframe.

In [202]:
def make_recommendation(product):
  df_final = merge_products_with_same_order_id()   # returns merged and cleaned dataframe.
  res = df_final[df_final['Product'].str.contains(str(product), case=False)].sort_values(by=['Order ID'],ascending = [False]).head(1)
  res = res['Product'].iloc[0]
  #Write code for recommendation
  to_remove = [str(product), ', ']
  for r in to_remove: 
    res = res.replace(r, '')
  return res


product = 'Google Phone'
print(make_recommendation(product))

USB-C Charging Cable


In [177]:
from tabulate import tabulate 

In [205]:
df_products = pd.DataFrame(df['Product'].unique(), columns=['Product Name'])
print(tabulate(df_products, headers = 'keys', tablefmt = 'psql'))
val = input("Enter the name of a product to get recommendation")
print("People buy: ",make_recommendation(val.strip()), "most with: ", val)

+----+----------------------------+
|    | Product Name               |
|----+----------------------------|
|  0 | USB-C Charging Cable       |
|  1 | Macbook Pro Laptop         |
|  2 | Flatscreen TV              |
|  3 | AAA Batteries (4-pack)     |
|  4 | Google Phone               |
|  5 | Wired Headphones           |
|  6 | LG Dryer                   |
|  7 | iPhone                     |
|  8 | Bose SoundSport Headphones |
|  9 | AA Batteries (4-pack)      |
| 10 | Lightning Charging Cable   |
| 11 | Vareebadd Phone            |
| 12 | Apple Airpods Headphones   |
| 13 | 20in Monitor               |
| 14 | 34in Ultrawide Monitor     |
| 15 | LG Washing Machine         |
| 16 | 27in 4K Gaming Monitor     |
| 17 | ThinkPad Laptop            |
| 18 | 27in FHD Monitor           |
+----+----------------------------+
Enter the name of a product to get recommendationApple Airpods Headphones
People buy:  iPhone most with:  Apple Airpods Headphones
